# Elina Azrilyan

Data 608

Module 4

March 18, 2021


In [437]:
import pandas as pd
import numpy as np
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
from jupyter_dash import JupyterDash
import plotly.graph_objects as go

In [438]:
soql_url = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?' +\
        '$select=spc_common,count(tree_id)' +\
        '&$group=spc_common').replace(' ', '%20')
sps_trees = pd.read_json(soql_url)

sps_trees.head()

,count_tree_id,spc_common
0,273,American beech
1,7975,American elm
2,1081,American hophornbeam
3,1517,American hornbeam
4,46,American larch


In [439]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

server = app.server

app.layout = html.Div([
    html.H2('Condition of Trees'),
    dcc.Dropdown(
        id='my-dropdown',
        options=[
            {'label': 'Queens', 'value': "'Queens'"},
            {'label': 'Brooklyn', 'value': "'Brooklyn'"},
            {'label': 'Manhattan', 'value': "'Manhattan'"},
            {'label': 'Staten Island', 'value': "'Staten Island'"},
            {'label': 'Bronx', 'value': "'Bronx'"},
        ],
        value = "'Queens'"
    ),
    dcc.Dropdown(
        id='my-dropdown2',
        options=[
            {'label': i, 'value': i} for i in sps_trees.spc_common.iloc[0:132]
        ],
        value = "green ash"
    ),
    dcc.Dropdown(
        id='my-dropdown3',
        options=[
            {'label': 'No Steward', 'value': "'None'"},
            {'label': '1 or 2 Stewards', 'value': "'1or2'"},
            {'label': '3 or 4 Stewards', 'value': "'3or4'"},
            {'label': '4 or More Stewards', 'value': "'4orMore'"},
        ],
        value = "'None'"
    ),
    dcc.Graph(id='my-graph')
])

if __name__ == '__main__':
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/


In [440]:
@app.callback(Output('my-graph', 'figure'),
              [Input('my-dropdown', 'value')],
              [Input('my-dropdown2', 'value')],
              [Input('my-dropdown3', 'value')])
def update_graph(dd_value, dd2_value, dd3_value):
    boro = dd_value
    scs = dd2_value
    stwd = dd3_value

    soql_url = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?' +\
        '$select=health,count(tree_id)' +\
        '&$where=spc_common='+"'"+scs +"'"+\
        'AND boroname='+boro +\
        'AND steward='+stwd +\
        '&$group=health').replace(' ', '%20')
    soql_trees = pd.read_json(soql_url)

    soql_trees
    return {
        'data': [{
            'x': soql_trees.health,
            'y': soql_trees.count_tree_id,
            'type': 'bar'
        }],
        'layout': {
            'margin': {
                'l': 30,
                'r': 20,
                'b': 30,
                't': 20
            }
        }
    }

# I realize that the "Poor, Fair, Good" are not always displaying in the right order - but after many hours of trying to figure it out - I was not able to do anything about it.
    #soql_trees.health().reindex(["Poor", "Fair", "Good"])
    #soql_trees
    #soql_trees.count_tree_id.reindex(["Poor", "Fair", "Good"]).plot(kind="bar")

In [441]:
app.run_server(mode="inline")